In [1]:
import warnings
warnings.filterwarnings("ignore")
from utils.io_utils import *
from utils.opensim_kinematics_utils import *
from utils.data_processing_utils import *
from utils.analysis_utils import *

In [2]:
# define variables

MODEL = "gait2354_simbody.osim"
ORIGINAL_MARKETSET_FILE = "example_markerset_files/HMB2_MODEL_markers.xml"
# list of the names of the markers in ORIGINAL_MARKERSET_FILE
ORIGINAL_MARKERS = [
    "LASIS", "RASIS", "LPSIS", "RPSIS", "LLTHI", "LLEK", "LMEK", "LLSHA", "LLM",
    "LMM", "LHEE", "LMT2", "LMT5", "RLTHI", "RLEK", "RMEK", "RLSHA", "RLM", "RMM",
    "RHEE", "RMT2", "RMT5", "C7", "T10", "XIPH", "JN"
    ]
C3D_CALIBRATION_FILE = "example_c3d_files/Calibration_Mathieu.c3d"
C3D_DYNAMIC_FILE = "example_c3d_files/Walk_Mathieu.c3d"
TEMPLATE_SCALING_SETUP_FILE = "template_scaling_setup.xml"

NEW_MARKERSET_FILE = "example_markerset_files/HMB2_MODEL_LUCILLE_markers.xml"
# list of the names of the markers in NEW_MARKERSET_FILE
NEW_MARKERS = ALL_MARKERS = ["C7", "T10", "STRN", "LASI", "RASI", "LPSI", "RPSI",
               "LTHI", "LKNE", "LMEK", "LANK", "LMM", "LHEE", "LTOE", "LMT2", "LMT5",
			   "RTHI", "RKNE", "RMEK", "RANK", "RMM", "RHEE", "RTOE", "RMT2", "RMT5",
               "RStatic1", "RStatic2", "RStatic3", "RStatic4", "RStatic5", "RStatic6", "RStatic7",
               "Rrear1", "Rrear2",
               "LStatic1", "LStatic2", "LStatic3", "LStatic4", "LStatic5",
			   "Lrear1", "Lrear2", "LMIMU",
               "pelvis1", "pelvis2", "pelvis3", "pelvis4"
               ]

RESULTS_FOLDER = "results_marker_translation"
PREFIX = "translation" # convention for naming of new created files

In [3]:
os.makedirs(RESULTS_FOLDER, exist_ok=True)  

In [4]:
# STEP 1: SCALE MODEL WITH .c3d CALIBRATION FILE
scaling(PREFIX,
        RESULTS_FOLDER,
        ORIGINAL_MARKERS,
        TEMPLATE_SCALING_SETUP_FILE,
        ORIGINAL_MARKETSET_FILE,
        c3d_calib_file = C3D_CALIBRATION_FILE)

path_to_scaled_model = os.path.join(RESULTS_FOLDER, PREFIX + "_scaled_model_markers.osim")

# STEP 2: INVERSE KINEMATICS WITH .c3d CALIBRATION and DYNAMIC FILE (BASED ON ORIGINAL MARKERSET) TO GET CORRESPONDING .mot file
# (this mot file gives the kinematics of the model while performing the calibration and dynamic tasks
# and will allow to perform point kineamtics to retrieve the coordinates of the new markerset)
(calib_start_time, calib_stop_time) = inverse_kinematics(PREFIX,
                   "calib",
                   ORIGINAL_MARKERS,
                   ORIGINAL_MARKETSET_FILE,
                   path_to_scaled_model,
                   RESULTS_FOLDER,
                   c3d_dynamic_file=C3D_CALIBRATION_FILE)

(walk_start_time, walk_stop_time) = inverse_kinematics(PREFIX,
                   "walk",
                   ORIGINAL_MARKERS,
                   ORIGINAL_MARKETSET_FILE,
                   path_to_scaled_model,
                   RESULTS_FOLDER,
                   c3d_dynamic_file=C3D_DYNAMIC_FILE) # here we do not specify start and stop times so the entire motion is processed

path_to_calib_motion_file = os.path.join(RESULTS_FOLDER, PREFIX+"_calib_motion.mot")
path_to_walk_motion_file = os.path.join(RESULTS_FOLDER, PREFIX+"_walk_motion.mot")


# STEP 3: POINT KINEMATICS WITH NEW MARKERSET ON BOTH .mot FILE RESULTING FROM THE PREVIOUS INVERSE KINEMATICS STEP
point_kinematics ("new_markerset_"+PREFIX,
                  "calib",
                  NEW_MARKERSET_FILE,
                  RESULTS_FOLDER,
                  path_to_scaled_model,
                  path_to_calib_motion_file,
                  start_time=calib_start_time,
                  end_time=calib_stop_time)

point_kinematics ("new_markerset_"+PREFIX,
                  "walk",
                  NEW_MARKERSET_FILE,
                  RESULTS_FOLDER,
                  path_to_scaled_model,
                  path_to_walk_motion_file,
                  start_time=walk_start_time,
                  end_time=walk_stop_time)

path_to_translated_calib_trc_file = os.path.join(RESULTS_FOLDER, "new_markerset_"+PREFIX+"_calib.trc")
path_to_translated_walk_trc_file = os.path.join(RESULTS_FOLDER, "new_markerset_"+PREFIX+"_walk.trc")

# STEP 4: CONVERT resulting .trc FILE RESULTING FROM PREVIOUS POINTKIN STEP INTO .c3d FILE
convert_trc_to_c3d(path_to_translated_calib_trc_file, source_c3d_path=C3D_CALIBRATION_FILE)
convert_trc_to_c3d(path_to_translated_walk_trc_file, source_c3d_path=C3D_DYNAMIC_FILE)

700
0 100
✅ Conversion terminée : results_marker_translation\translation_calib_static.trc
700
0 700
✅ Conversion terminée : results_marker_translation\translation_calib.trc
8031
0 8031
✅ Conversion terminée : results_marker_translation\translation_walk.trc
TRC file saved: results_marker_translation\new_markerset_translation_calib.trc
TRC file saved: results_marker_translation\new_markerset_translation_walk.trc
🔄 Ajout des données de force depuis : example_c3d_files/Calibration_Mathieu.c3d
💾 Écriture du fichier C3D : results_marker_translation\new_markerset_translation_calib.c3d
✅ Fichier C3D sauvegardé : results_marker_translation\new_markerset_translation_calib.c3d
🔄 Ajout des données de force depuis : example_c3d_files/Walk_Mathieu.c3d
💾 Écriture du fichier C3D : results_marker_translation\new_markerset_translation_walk.c3d
✅ Fichier C3D sauvegardé : results_marker_translation\new_markerset_translation_walk.c3d
